In [1]:
from os import listdir
from os.path import isfile, join

statdir = "playerstats"
files = [join(statdir, f) for f in listdir(statdir) if isfile(join(statdir, f))]

In [2]:
import json

playerdata = []
for jf in files:
    with open(jf) as f:
        playerdata.append(json.load(f))

In [3]:
playerdata[0]

[{u'Assists': 8,
  u'Deaths': 4,
  u'GameDuration': 1277,
  u'GoldEarned': 8509,
  u'Kills': 3,
  u'Lane': u'TOP',
  u'MatchID': 2081883619,
  u'Role': u'SOLO',
  u'Winner': True},
 {u'Assists': 8,
  u'Deaths': 4,
  u'GameDuration': 1250,
  u'GoldEarned': 9377,
  u'Kills': 5,
  u'Lane': u'TOP',
  u'MatchID': 2084375383,
  u'Role': u'SOLO',
  u'Winner': True},
 {u'Assists': 4,
  u'Deaths': 11,
  u'GameDuration': 1984,
  u'GoldEarned': 12639,
  u'Kills': 4,
  u'Lane': u'TOP',
  u'MatchID': 2084988182,
  u'Role': u'SOLO',
  u'Winner': False},
 {u'Assists': 3,
  u'Deaths': 7,
  u'GameDuration': 1705,
  u'GoldEarned': 9531,
  u'Kills': 3,
  u'Lane': u'TOP',
  u'MatchID': 2085158522,
  u'Role': u'SOLO',
  u'Winner': False},
 {u'Assists': 5,
  u'Deaths': 9,
  u'GameDuration': 1377,
  u'GoldEarned': 5863,
  u'Kills': 1,
  u'Lane': u'BOTTOM',
  u'MatchID': 2085252893,
  u'Role': u'DUO_SUPPORT',
  u'Winner': False},
 {u'Assists': 17,
  u'Deaths': 4,
  u'GameDuration': 1514,
  u'GoldEarned': 8440

In [4]:
import numpy as np

ignorethese = ('MatchID', 'Lane', 'Role')

def filterdata(data, lane, role):
    return filter(lambda stat: (lane == "ANY" or stat["Lane"] == lane) and 
                                    (role == "ANY" or stat["Role"] == role), 
                                   data)

def createAvgStats(stats, lane, role):
    avgstats = {}
    filteredstats = filterdata(stats, lane, role)
    if len(filteredstats) == 0:
        return None
    for stat in filteredstats:
        for key in stat:
            if key not in ignorethese:
                if key in avgstats:
                    avgstats[key] += stat[key]
                else:
                    avgstats[key] = stat[key]
    for key in avgstats:
        avgstats[key] /= float(len(filteredstats))
    avgstats["Lane"] = lane
    avgstats["Role"] = role
    return avgstats

def createPlayersAvgStats(playerdata, lane, role):
    arr = []
    for p in playerdata:
        ap = createAvgStats(p, lane, role)
        if ap:
            ap["GamesPlayed"] = len(filterdata(p, lane, role))
            arr.append(ap)
    return createAvgStats(arr, lane, role)

def createStatsDistribution(stats, lane, role):
    statdistribution = {}
    filteredstats = filterdata(stats, lane, role)
    if len(filteredstats) == 0:
        return None
    for stat in filteredstats:
        for key in stat:
            if key not in ignorethese:
                if key in statdistribution:
                    statdistribution[key].append(stat[key])
                else:
                    statdistribution[key] = [stat[key]]
        
    statdistribution["Lane"] = lane
    statdistribution["Role"] = role
    return statdistribution

def createStdStats(stats, lane, role):
    sd = createStatsDistribution(stats, lane, role)
    
    if sd == None:
        return None
    
    stds = {}
    for key in sd:
        if key == "Lane" or key == "Role":
            stds[key] = sd[key]
        else:
            stds[key] = np.std(sd[key])
    
    return stds

def createPlayersStdStats(playerdata, lane, role):
    arr = []
    for p in playerdata:
        ap = createAvgStats(p, lane, role)
        if ap:
            ap["GamesPlayed"] = len(filterdata(p, lane, role))
            arr.append(ap)
    
    return createStdStats(arr, lane, role)

In [5]:
avgstatdata = []
avgstatdata.append(createPlayersAvgStats(playerdata, "TOP", "SOLO"))
avgstatdata.append(createPlayersAvgStats(playerdata, "JUNGLE", "NONE"))
avgstatdata.append(createPlayersAvgStats(playerdata, "MIDDLE", "SOLO"))
avgstatdata.append(createPlayersAvgStats(playerdata, "BOTTOM", "DUO_CARRY"))
avgstatdata.append(createPlayersAvgStats(playerdata, "BOTTOM", "DUO_SUPPORT"))
avgstatdata.append(createPlayersAvgStats(playerdata, "ANY", "ANY"))

In [6]:
with open('player_stats_mean.json', 'w') as f:
    json.dump(avgstatdata, f)

In [7]:
stddata = []
stddata.append(createPlayersStdStats(playerdata, "TOP", "SOLO"))
stddata.append(createPlayersStdStats(playerdata, "JUNGLE", "NONE"))
stddata.append(createPlayersStdStats(playerdata, "MIDDLE", "SOLO"))
stddata.append(createPlayersStdStats(playerdata, "BOTTOM", "DUO_CARRY"))
stddata.append(createPlayersStdStats(playerdata, "BOTTOM", "DUO_SUPPORT"))
stddata.append(createPlayersStdStats(playerdata, "ANY", "ANY"))

In [8]:
with open('player_stats_std.json', 'w') as f:
    json.dump(stddata, f)

In [10]:
avgstatdata

[{u'Assists': 6.566713345850645,
  u'Deaths': 5.142151140373734,
  u'GameDuration': 1774.550740470847,
  'GamesPlayed': 27.381738173817382,
  u'GoldEarned': 11657.409759441523,
  u'Kills': 5.102016583078837,
  'Lane': 'TOP',
  'Role': 'SOLO',
  u'Winner': 0.5146784553531175},
 {u'Assists': 8.03479913032049,
  u'Deaths': 4.589331060080931,
  u'GameDuration': 1750.017780298054,
  'GamesPlayed': 25.317277486910996,
  u'GoldEarned': 11447.10608285148,
  u'Kills': 6.058622115672339,
  'Lane': 'JUNGLE',
  'Role': 'NONE',
  u'Winner': 0.530269532093439},
 {u'Assists': 6.962195894657523,
  u'Deaths': 4.9392847918767675,
  u'GameDuration': 1778.9076856066804,
  'GamesPlayed': 22.980710659898477,
  u'GoldEarned': 11994.429165883143,
  u'Kills': 6.179187274629288,
  'Lane': 'MIDDLE',
  'Role': 'SOLO',
  u'Winner': 0.5262707465125209},
 {u'Assists': 6.877645342704994,
  u'Deaths': 5.083456375166942,
  u'GameDuration': 1790.2761296279107,
  'GamesPlayed': 24.74908200734394,
  u'GoldEarned': 12279.1

In [11]:
stddata

[{u'Assists': 2.4610617019823917,
  u'Deaths': 1.6630919197583729,
  u'GameDuration': 218.77471279932558,
  'GamesPlayed': 44.295916328250733,
  u'GoldEarned': 1861.0586695714953,
  u'Kills': 2.075805353735106,
  'Lane': 'TOP',
  'Role': 'SOLO',
  u'Winner': 0.26024268252757815},
 {u'Assists': 3.0106214108139788,
  u'Deaths': 1.5149239345572993,
  u'GameDuration': 247.90905440851259,
  'GamesPlayed': 37.163609754302946,
  u'GoldEarned': 2147.6381013162436,
  u'Kills': 2.44797443686587,
  'Lane': 'JUNGLE',
  'Role': 'NONE',
  u'Winner': 0.25539806215614025},
 {u'Assists': 2.2817256134544213,
  u'Deaths': 1.5281760929842354,
  u'GameDuration': 191.42117205427729,
  'GamesPlayed': 32.104092019056679,
  u'GoldEarned': 1635.980280368942,
  u'Kills': 2.2600782060855162,
  'Lane': 'MIDDLE',
  'Role': 'SOLO',
  u'Winner': 0.2382878285369068},
 {u'Assists': 2.2696230009859319,
  u'Deaths': 1.6164334095750736,
  u'GameDuration': 224.40953249236512,
  'GamesPlayed': 35.949569631427146,
  u'GoldEa

In [118]:
distdata = createStatsDistribution(stats, "ANY", "ANY")

In [18]:
allgames = [item for sublist in playerdata for item in sublist]

In [22]:
print len(filterdata(allgames, "TOP", "SOLO"))
print len(filterdata(allgames, "TOP", "ANY"))
print len(filterdata(allgames, "MIDDLE", "SOLO"))
print len(filterdata(allgames, "MIDDLE", "ANY"))

24890
24973
22636
23248


In [23]:
len(allgames)

113195